In [2]:
import numpy as np, pandas as pd
from datascience import *
tl = Table.read_table("TL_with_dist_10_10_JamesEdits.csv")

In [3]:
tl

Site,Participant_ID,Q69,Date_bound_lower,Date_bound_upper,Trip_number,Trip_ID,Q2_1,Q2_2,Q2_3,Q3,Q3_1_TEXT,Q3_2_TEXT,Q65,Q5_1,Q5_2,Q5_3,Q10,Q10_9_TEXT,Q6,Q6_1_TEXT,Q6_2_TEXT,Q7,Q7_7_TEXT,Q8,State,Origin_Latitude,Origin_Longitude,Origin_Accuracy Score,Origin_Accuracy Type,Origin_Number,Origin_Street,Origin_City,Origin_State,Origin_County,Origin_Zip,Origin_Country,Origin_Source,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,City,State.1,County,Zip,Country,Source,TOD?,Placetype,on e-mission server,Date onboarded,Time onboarded,GSR,RA onboarded,Installed App? (y/n),Onboarding notes,Offboarded? (1 = yes),Date offboarded,Time offboarded,GSR.1,RA offboarded,Downloaded app data?,Offboarding notes,GSR Lead,Carol has contact?,Status,Within Half Mile of HQT? (TOD?),PDX Place Type,Lat,Long,Notes,Contact Name,Phone # or email,City_2,Units with Subsidy,PIS,Management Company,Developer,Community room we can use?,Wifi?,Language Needs,Total # Participants,Double Check Participants_Survey,Travel Log Total,GSR Survey Lead,Survey completed?,Outreach touch point 1,Outreach touch point 2,Outreach touch point 3,Notes.1,Drove_Alone,Drove_Share,Bus,Train,Shuttle,Walk,Bike,Taxi/Uber/Lyft,Transit,Drove_all,Vehicle,Home,Work,Education,Shopping,Meal,Personal/Medical,Social/Recreational,Pick_up_drop_off,URL,Distance
Alta Mira,Alta1,4/4/2018,nan,nan,1,210,Alta Mira,28939 Mission Blvd.,Hayward,PM:,nan,4:00,nan,GSO (work),4601 Malat St,Oakland,Work,nan,PM:,nan,6:00,"Bus,Train",nan,0,CA,37.6351,-122.053,1,rooftop,28939,Mission Blvd,Hayward,CA,Alameda County,94544,US,City of Hayward and Fairview. Licensed for Public Use,37.7648,-122.221,1,rooftop,4601,Malat St,Oakland,CA,Alameda County,94601,US,Alameda County,1,Urban Neighborhood,0,30-Mar,3:30-5:30PM,James,nan,N,nan,1,4/6/2018,3:30-5pm,Rebecca,Juan,nan,nan,James/Rebecca,nan,Data Collection scheduled for 3/30,Yes,Urban Neighborhood,37.6349,-122.054,nan,nan,shira.goldsmith@edenhousing.org,nan,150,nan,nan,Eden,nan,nan,nan,10,10,nan,Rebecca,Y,16-Apr,nan,nan,nan,0,0,1,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,https://maps.googleapis.com/maps/api/distancematrix/json ...,23.8
Alta Mira,Alta1,4/4/2018,nan,nan,2,499,GSO (work),4601 Malat St,Oakland,PM:,nan,10:00,nan,Alta Mira,28939 Mission Blvd.,Hayward,Home,nan,AM:,10:45,nan,"Bus,Train",nan,0,CA,37.7648,-122.221,1,rooftop,4601,Malat St,Oakland,CA,Alameda County,94601,US,Alameda County,37.6351,-122.053,1,rooftop,28939,Mission Blvd,Hayward,CA,Alameda County,94544,US,City of Hayward and Fairview. Licensed for Public Use,1,Urban Neighborhood,0,30-Mar,3:30-5:30PM,James,nan,N,nan,1,4/6/2018,3:30-5pm,Rebecca,Juan,nan,nan,James/Rebecca,nan,Data Collection scheduled for 3/30,Yes,Urban Neighborhood,37.6349,-122.054,nan,nan,shira.goldsmith@edenhousing.org,nan,150,nan,nan,Eden,nan,nan,nan,10,10,nan,Rebecca,Y,16-Apr,nan,nan,nan,0,0,1,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,https://maps.googleapis.com/maps/api/distancematrix/json ...,23.8
Alta Mira,Dn22QF,3/31/2018,nan,nan,1,214,Home,28939 mission Blv,Hayward,PM:,nan,4:23,nan,nan,Holly St and 96th St,Oakland,Social/Recreational,nan,PM:,nan,4:48,Drove alone,nan,4,CA,37.6346,-122.053,1,rooftop,28939,Mission Bl,Hayward,CA,Alameda County,94544,US,Alameda County,37.7463,-122.17,1,intersection,nan,96th Ave,Oakland,CA,Alameda County,94603,US,TIGER/Line® dataset from the US Census Bureau,1,Urban Neighborhood,1,30-Mar,3:30-5:30PM,James,nan,N,nan,1,4/6/2018,3:30-5pm,Rebecca,Jenny,nan,nan,James/Rebecca,nan,Data Collection scheduled for 3/30,Yes,Urban Neighborhood,37.6349,-122.054,nan,nan,shira.goldsmith@edenhousing.org,nan,150,nan,nan,Eden,nan,nan,nan,10,10,nan,Rebecca,Y,16-Apr,nan,nan,nan,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,https://maps.googleapis.com/maps/api/distancematrix/json ...,19
Alta Mira,LapWx5,5/4/2018,nan,nan,1,206,Home,28939 Mission Blvd,Hayward,AM:,8,nan,nan,Global Supply,20628 Corsair bld,Hayward,Work,nan,AM:,8:15,nan,Drove alone,nan,0,CA,37.6351,-122.053,1,rooftop,28939,Mission Blvd,Hayward,CA,Alameda County,94544,US,City of Hayw

In [4]:
tod = tl.where("Within Half Mile of HQT? (TOD?)", "Yes")
control = tl.where("Within Half Mile of HQT? (TOD?)", "No")

In [5]:
tod.group('PDX Place Type')

PDX Place Type,count
Suburban Neighborhood,117
Urban Core,348
Urban District,195
Urban Neighborhood,246


In [6]:
control.group('PDX Place Type')

PDX Place Type,count
Suburban Neighborhood,337
Urban Neighborhood,96


In [7]:
suburban = tl.where('PDX Place Type','Suburban Neighborhood')
urban = tl.where('PDX Place Type', are.not_equal_to('Suburban Neighborhood'))
tod_urban = tod.where('PDX Place Type', are.not_equal_to('Suburban Neighborhood'))
control_urban = control.where('PDX Place Type', are.not_equal_to('Suburban Neighborhood'))
tod_sub = tod.where('PDX Place Type','Suburban Neighborhood')
control_sub = control.where('PDX Place Type','Suburban Neighborhood')

In [8]:
def drive_percent(tbl):
    return(len(tbl.where('Drove_all',1).column(0))/len(tbl.column(0)))
def carpool_percent(tbl):
    return(len(tbl.where('Drove_Share',1).column(0))/len(tbl.where('Drove_all',1).column(0)))
def work_drive(tbl):
    return(len(tbl.where('Work',1).where('Drove_all',1).column(0))/len(tbl.where('Work',1).column(0)))
def rec_drive(tbl):
    return(len(tbl.where('Social/Recreational',1).where('Drove_all',1).column(0))/len(tbl.where('Social/Recreational',1).column(0)))
def mean_dist(table):
    return np.mean(table.where('Distance',are.below(9999)).column('Distance'))
def mean_car(table):
    return np.mean(table.where('Distance',are.below(9999)).where('Drove_all',1).column('Distance'))
def mean_transit(table):
    return np.mean(table.where('Distance',are.below(9999)).where('Transit',1).column('Distance'))
def mean_walk(table):
    return np.mean(table.where('Distance',are.below(9999)).where('Walk',1).where('Transit',0).column('Distance'))

In [72]:
types = [tod_urban,control_urban,tod_sub,control_sub]
labels = make_array('tod_urban','control_urban','tod_sub','control_sub')
def agg_table_gen(types,labels):
    drive = make_array()
    carpool = make_array()
    work = make_array()
    rec = make_array()
    dist = make_array()
    car_dist = make_array()
    tran_dist = make_array()
    walk_dist = make_array()
    for i in types:
        drive = np.append(drive,drive_percent(i))
        carpool = np.append(carpool,carpool_percent(i))
        work = np.append(work,work_drive(i))
        rec = np.append(rec,rec_drive(i))
        dist = np.append(dist, mean_dist(i))
        car_dist = np.append(car_dist, mean_car(i))
        tran_dist = np.append(tran_dist, mean_transit(i))
        walk_dist = np.append(walk_dist, mean_walk(i))
    agg_table = Table().with_columns('Housing Type',labels,'Percent Trips Driving',np.round(drive*100,1),'Percent Driving Trips Carpooled',np.round(carpool*100,1),'Percent Work Trips Driving',np.round(work*100,1),'Percent Recreation Trips Driving',np.round(rec*100,1),'Mean Distance of All Trips (km)',np.round(dist,2),'Mean Distance of Driving Trips (km)',np.round(car_dist,2),'Mean Distance of Transit Trips (km)',np.round(tran_dist,2),'Mean Distance of Walking Trips (km)',np.round(walk_dist,2))    
    return agg_table
agg_table_gen(types,labels)

Housing Type,Percent Trips Driving,Percent Driving Trips Carpooled,Percent Work Trips Driving,Percent Recreation Trips Driving,Mean Distance of All Trips (km),Mean Distance of Driving Trips (km),Mean Distance of Transit Trips (km),Mean Distance of Walking Trips (km)
tod_urban,52.7,24.5,62.9,51.6,7.26,9.16,8.94,1.28
control_urban,60.4,13.8,50,66.7,5.28,5.83,6.78,1.25
tod_sub,72.6,29.4,68.8,71.4,6.89,6.82,22.16,0.86
control_sub,77.7,27.1,85.7,63.6,7.83,7.85,12.87,0.93


In [73]:
types2 = [control,tod]
labels2 = ['control','tod']
agg_table_gen(types2,labels2)

Housing Type,Percent Trips Driving,Percent Driving Trips Carpooled,Percent Work Trips Driving,Percent Recreation Trips Driving,Mean Distance of All Trips (km),Mean Distance of Driving Trips (km),Mean Distance of Transit Trips (km),Mean Distance of Walking Trips (km)
control,73.9,24.7,75.9,64.3,7.24,7.47,10.48,1.02
tod,55.3,25.3,64,53.6,7.22,8.78,9.66,1.24


In [74]:
types3 = [suburban,urban]
labels3 = ['suburban','urban']
agg_table_gen(types3,labels3)

Housing Type,Percent Trips Driving,Percent Driving Trips Carpooled,Percent Work Trips Driving,Percent Recreation Trips Driving,Mean Distance of All Trips (km),Mean Distance of Driving Trips (km),Mean Distance of Transit Trips (km),Mean Distance of Walking Trips (km)
suburban,76.4,27.7,78.4,66.7,7.58,7.6,14.96,0.9
urban,53.6,23.2,61.5,52.3,7.05,8.75,8.69,1.28


In [10]:
np.mean(tl.where('Distance',are.below(9999)).column('Distance'))

7.223101460415066

In [11]:
number_of_trips = make_array()
for i in types:
    number_of_trips = np.append(number_of_trips, i.num_rows)
number_of_participants = make_array()
for i in types:
    number_of_participants = np.append(number_of_participants, i.group('Participant_ID').num_rows)
avg_num_trips = number_of_trips/number_of_participants
sum_table = Table().with_columns('Neighborhood Type',labels,'Total number of trips', number_of_trips,'Number of participants', number_of_participants,'Average trips per participant',np.round(avg_num_trips,2))
sum_table

Neighborhood Type,Total number of trips,Number of participants,Average trips per participant
tod_urban,789,182,4.34
control_urban,96,19,5.05
tod_sub,117,24,4.88
control_sub,337,69,4.88


In [75]:
def transit_breakdown(mode):
    mode_array = make_array()
    for i in types:
        transit = i.where('Transit',1)
        mode_tot = sum(i.column(mode))
        mode_array = np.append(mode_array,mode_tot/transit.num_rows)
    return mode_array
tran_table = Table().with_columns('Housing Type',labels,'Bus Percent',np.round(transit_breakdown('Bus')*100,2),'Train Percent',np.round(transit_breakdown('Train')*100,2),'Shuttle Percent',np.round(transit_breakdown('Shuttle')*100,2))
tran_table

Housing Type,Bus Percent,Train Percent,Shuttle Percent
tod_urban,73.72,41.67,2.56
control_urban,85,15,0
tod_sub,55.56,55.56,11.11
control_sub,87.1,12.9,6.45


## Statistical Significance Testing

In [ ]:
#create a 

In [51]:
def z_score(group1,group2,table_stat,function):
    size1 = group1.num_rows
    size2 = group2.num_rows
    sd1 = np.std(group1.column(table_stat))
    sd2 = np.std(group2.column(table_stat))
    se_diff = ((sd1/(size1**.5))**2+(sd2/(size2**.5))**2)**.5
    value1 = function(group1)
    value2 = function(group2)
    return (value2-value1)/se_diff
z_score(tod,control,'Drove_all',drive_percent)

6.942087782602879

In [78]:
z_score(urban,suburban,'Drove_all',drive_percent)

8.78518802620414

In [81]:
z_score(tod_urban,control_urban,'Drove_all',drive_percent)

1.4517667329801647

In [97]:
def z_matrix(table_stat,function):
    z_table = Table().with_column('Type',labels)
    for i in np.arange(len(labels)):
        z_values =[]
        for j in types:
            z_values = np.append(z_score(types[i],j,table_stat,function),z_values)
        z_table = z_table.with_column(labels[i],z_values[::-1])
    return z_table
z_matrix('Drove_all',drive_percent)

Type,tod_urban,control_urban,tod_sub,control_sub
tod_urban,0,-1.45177,-4.43954,-8.68799
control_urban,1.45177,0,-1.88996,-3.16127
tod_sub,4.43954,1.88996,0,-1.08343
control_sub,8.68799,3.16127,1.08343,0


In [98]:
z_matrix('Distance',mean_dist)

Type,tod_urban,control_urban,tod_sub,control_sub
tod_urban,0,0.0176775,0.00192238,-0.00417774
control_urban,-0.0176775,0,-0.0075697,-0.0154628
tod_sub,-0.00192238,0.0075697,0,-0.00412659
control_sub,0.00417774,0.0154628,0.00412659,0


In [99]:
z_matrix('Drove_Share',carpool_percent)

Type,tod_urban,control_urban,tod_sub,control_sub
tod_urban,0,3.50149,-1.23135,-1.02294
control_urban,-3.50149,0,-3.30612,-3.70592
tod_sub,1.23135,3.30612,0,0.526458
control_sub,1.02294,3.70592,-0.526458,0


In [19]:
agg_table.to_csv('TOD_type_agg_table.csv')

In [24]:
sum_table.to_csv('TOD_type_sum_table.csv')

In [25]:
tran_table.to_csv('TOD_type_tran_table.csv')